# RFM Analysis on Online Retail (UCI)
This notebook loads the **OnlineRetail.csv** file, cleans it, computes **Recency, Frequency, Monetary (RFM)** metrics, scores customers, creates simple customer segments, and visualizes the results.

In [ ]:
# Step 0: Setup
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Display options
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 120)

# Load dataset (already uploaded to this environment)
path = '/mnt/data/OnlineRetail.csv'
df = pd.read_csv(path, encoding='ISO-8859-1')

print('Rows:', len(df))
df.head()

In [ ]:
# Step 1: Data Cleaning
# 1) Remove rows with missing CustomerID
# 2) Remove cancelled invoices (InvoiceNo that start with 'C')
# 3) Convert InvoiceDate to datetime
# 4) Create TotalPrice = Quantity * UnitPrice

df_clean = df.dropna(subset=['CustomerID']).copy()
df_clean = df_clean[~df_clean['InvoiceNo'].astype(str).str.startswith('C')].copy()
df_clean['InvoiceDate'] = pd.to_datetime(df_clean['InvoiceDate'])
df_clean['TotalPrice'] = df_clean['Quantity'] * df_clean['UnitPrice']

df_clean.head()

In [ ]:
# Step 2: Calculate R, F, M
latest_date = df_clean['InvoiceDate'].max() + pd.Timedelta(days=1)

rfm = (df_clean
       .groupby('CustomerID')
       .agg(Recency=('InvoiceDate', lambda x: (latest_date - x.max()).days),
            Frequency=('InvoiceNo', 'nunique'),
            Monetary=('TotalPrice', 'sum'))
       .reset_index())

rfm.head()

In [ ]:
# Step 3: RFM Scoring (1-4)
# Use rank-based quantiles to avoid qcut bin-edge duplication on low-cardinality data

# For Recency: lower is better -> R score 4 (best) to 1 (worst)
rfm['R_Score'] = pd.qcut(rfm['Recency'].rank(method='first'), 4, labels=[4, 3, 2, 1]).astype(int)

# For Frequency & Monetary: higher is better -> score 1 (worst) to 4 (best)
rfm['F_Score'] = pd.qcut(rfm['Frequency'].rank(method='first'), 4, labels=[1, 2, 3, 4]).astype(int)
rfm['M_Score'] = pd.qcut(rfm['Monetary'].rank(method='first'), 4, labels=[1, 2, 3, 4]).astype(int)

rfm['RFM_Segment'] = (rfm['R_Score'].astype(str) + rfm['F_Score'].astype(str) + rfm['M_Score'].astype(str))
rfm['RFM_Score'] = rfm[['R_Score','F_Score','M_Score']].sum(axis=1)

rfm[['CustomerID','Recency','Frequency','Monetary','R_Score','F_Score','M_Score','RFM_Segment','RFM_Score']].head()

In [ ]:
# Step 4: Simple Customer Segments (you can tune thresholds)
def segment_row(row):
    if row['RFM_Score'] >= 9:
        return 'Best Customers'
    elif row['RFM_Score'] >= 8:
        return 'Loyal Customers'
    elif row['RFM_Score'] >= 6:
        return 'Potential Loyalists'
    elif row['RFM_Score'] >= 5:
        return 'At Risk'
    else:
        return 'Hibernating'

rfm['Segment'] = rfm.apply(segment_row, axis=1)

# Segment counts
seg_counts = rfm['Segment'].value_counts().rename_axis('Segment').reset_index(name='Count')
seg_counts

In [ ]:
# Step 5: Visualization - Segment Distribution (matplotlib only)
plt.figure(figsize=(10,6))
order = seg_counts.sort_values('Count', ascending=False)
plt.bar(order['Segment'], order['Count'])
plt.title('Customer Segments Distribution')
plt.xlabel('Segment')
plt.ylabel('Customers')
plt.xticks(rotation=30, ha='right')
plt.tight_layout()
plt.show()

In [ ]:
# Step 6 (Bonus): Heatmap of mean R, F, M by Segment (matplotlib)
stats = (rfm.groupby('Segment')[['Recency','Frequency','Monetary']]
         .mean()
         .reindex(order['Segment']))

fig = plt.figure(figsize=(6,4))
im = plt.imshow(stats.values, aspect='auto')
plt.colorbar(im, fraction=0.046, pad=0.04)
plt.yticks(range(len(stats.index)), stats.index)
plt.xticks(range(3), ['Recency','Frequency','Monetary'])
plt.title('Average R, F, M by Segment')
plt.tight_layout()
plt.show()

stats

In [ ]:
# Step 7: Save results
rfm_out = '/mnt/data/rfm_results.csv'
rfm.to_csv(rfm_out, index=False)
print('Saved:', rfm_out)